In [14]:
import numpy as np
from keras.models import Sequential
from keras.models import Model
from keras.layers import Dense
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.merge import concatenate
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
#
## visualize model
from keras.utils.vis_utils import plot_model

### Prepare Data

In [2]:
dataset = np.arange(10, 100, 10)
print(dataset)

[10 20 30 40 50 60 70 80 90]


In [3]:
def split_sequence(series, n_step, pred_step):
    x_list, y_list = [], [] 
    for s in range(series.shape[0]-(n_step+pred_step-1)):
        x_list.append(series[s:s+n_step]),
        y_list.append(series[s+n_step:s+n_step+pred_step])
    X, y = np.array(x_list), np.array(y_list)
    return X, y

In [4]:
n_step = 3
pred_step = 1
X, y = split_sequence(dataset, n_step, pred_step)
for i in range(X.shape[0]):
    print(X[i], y[i])

[10 20 30] [40]
[20 30 40] [50]
[30 40 50] [60]
[40 50 60] [70]
[50 60 70] [80]
[60 70 80] [90]


#### Our split sequence() function in the previous section outputs the X with the shape [samples, timesteps]

In [5]:
X.shape

(6, 3)

#### the model will expect the input component of training data to have the dimensions or shape: [samples, timesteps, features].

In [12]:
n_feature = 1
X_cnn = X.reshape(X.shape[0], X.shape[1], n_feature)
print(X_cnn.shape)
print(X_cnn)

(6, 3, 1)
[[[10]
  [20]
  [30]]

 [[20]
  [30]
  [40]]

 [[30]
  [40]
  [50]]

 [[40]
  [50]
  [60]]

 [[50]
  [60]
  [70]]

 [[60]
  [70]
  [80]]]


- The CNN does not actually view the data as having time steps, instead, it is treated as a sequence over which convolutional read operations can be performed, like a one-dimensional image. 
- In this example, we define a convolutional layer with 64 filter maps and a kernel size of 2.
- Sequence of 3 - parsed by kernal size of 2 -> resulting 2
- This is followed by a max pooling layer and a dense layer to interpret the input feature.
- An output layer is specified that predicts a single numerical value.
- The model is fit using the efficient Adam version of stochastic gradient descent and optimized using the mean squared error, or ‘mse’, loss function.

#### Max Pooling
<img src="./7-21-2019-max-pooling.png" alt="Drawing" style="width: 800px;"/>

In [13]:
print(n_step)
print(n_feature)

3
1


In [7]:
# define model
model = Sequential()
model.add(Conv1D(64, 2, activation='relu', input_shape=(n_step, n_feature)))
model.add(MaxPooling1D())
model.add(Flatten())
model.add(Dense(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 2, 64)             192       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1, 64)             0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                3250      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 3,493
Trainable params: 3,493
Non-trainable params: 0
_________________________________________________________________


- Input shape = (6, 3, 1)
- Conv1D(64, 2, activation='relu', input_shape=(n_step, n_feature)) -> kernel size = 2 -> 3 parsed by 2 = 2
- output shape = (none, 2, 64)
- MaxPooling1D() - max of 2 = 1 -> output shape = (None, 1, 64)
- Flatten input shape=(None, 1, 64) -> output_shape=((None, 64)

Param # 192 = 2 x 64 + 64 (bias) = 192
Param # 3250 = 64 x 50 + 50 (bias) = 3250
Param # 51 = 50 + 1 (bias) = 50

In [9]:
# model filt
model.fit(X_cnn, y, epochs=1000, verbose=0)

Instructions for updating:
Use tf.cast instead.


In [10]:
x_test = np.array([70, 80, 90])
X_test_cnn = x_test.reshape(1, n_step, n_feature)
print(X_test_cnn)
yhat = model.predict(X_test_cnn)

[[[70]
  [80]
  [90]]]


In [11]:
yhat

array([[104.34442]], dtype=float32)

In [15]:
plot_model(model, to_file='8_2_cnn.png', show_shapes=True, show_layer_names=True)

### Univariate CNN model
<img src="./8_2_cnn.png" alt="Drawing" style="width: 600px;"/>